In [1]:
import os
import pickle
import random
from tqdm import tqdm
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from PIL import Image
from torchvision.transforms import Resize
import torch.nn.functional as F


import re

In [2]:
data_dir = "S:/Capston/data2"


SAVE_FOLDER = "S:/_tmp/"

In [3]:
series_meta_df = pd.read_csv(os.path.join(data_dir,'train_series_meta.csv'))

In [4]:
series_meta_df.shape

(4711, 4)

In [15]:
# import csv
# # Write the list of .png files to a CSV file
# with open(os.path.join(data_dir,"images_file_name.csv"), 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(list_img)

In [16]:
list_img =os.listdir(os.path.join(data_dir,'train_images/'))
series_meta_df = pd.read_csv(os.path.join(data_dir,'train_series_meta.csv'))                             
lenght = series_meta_df.shape[0]     
img_paths = []
for index, row in series_meta_df.iterrows():
    scans = []
    patient = row["patient_id"]
    series = row["series_id"]
    pattern = "^" + str(int(patient)) + "_"+ str(int(series)) + "_"
    series_img_list  = [file for file in list_img if bool(re.match(pattern, file))]
    list_img = [file for file in list_img if file not in series_img_list ]
    series_img_paths = [os.path.join(data_dir, 'train_images/') + file for file in series_img_list]
    print(f"{index+1} / {lenght}, images in series:{len(series_img_paths)}", end='\r')
    img_paths.append(series_img_paths)

KeyboardInterrupt: 

In [ ]:
# import csv
# # Write the list of .png files to a CSV file
# with open(os.path.join(data_dir,"img_paths.csv"), 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(img_paths)

In [9]:
# import csv

# # Define the path to the CSV file
# csv_file_path = os.path.join(data_dir, "img_paths.csv")

# # Initialize a list to store the data from the CSV file
# img_paths = []

# # Read the CSV file
# with open(csv_file_path, 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         img_paths.append(row)

In [ ]:
len(img_paths)

In [ ]:
def Series_to_tensor(paths):
    series_tensor = torch.zeros(store_res, store_res, len(paths))
    for index, path in enumerate(paths):
        image = Image.open(path)
        image = np.array(image)
        image = torch.tensor(image, dtype=torch.float32)

        #resize the 2d image 
        if image.shape != torch.Size([store_res, store_res]):
            image = image.unsqueeze(0).unsqueeze(0)
            image = F.interpolate(image, (store_res, store_res))
            image = image.squeeze(0).squeeze(0)
        series_tensor[:,:,index] = image

    # Resize the 3d image along the vertical dimension.            
    series_tensor = series_tensor.unsqueeze(0).unsqueeze(0)
    series_tensor = F.interpolate(series_tensor, (store_res, store_res, store_res))
    series_tensor = series_tensor.squeeze(0).squeeze(0)
    return series_tensor

In [19]:
# Defining resolution
store_res = 180
Root = "S:/Capston/data3" 
# Iterate over rows with tqdm progress bar
for index, row in tqdm(series_meta_df.iterrows(), total=len(series_meta_df)):
    
    patient = row["patient_id"]
    series = row["series_id"]

    series_tensor = Series_to_tensor(img_paths[index])
    torch.save(series_tensor, os.path.join(Root, str(int(patient)) + '_' + str(int(series)) + '.pt'))

100%|████████████████████████████████████████████████████████████████████████████| 4711/4711 [7:55:24<00:00,  6.05s/it]


In [10]:

Root = "S:\\Capston\\data3"
series_meta_df["paths"] = series_meta_df.apply(lambda row: os.path.join(Root, f"{int(row['patient_id'])}_{int(row['series_id'])}.pt"), axis=1)

In [26]:
train_df = pd.read_csv(os.path.join(data_dir,'train.csv'))
series_meta_df = series_meta_df.merge(train_df, on='patient_id')

In [27]:
series_meta_df.head()

,patient_id,series_id,aortic_hu,incomplete_organ,paths,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,21057,146.00,0,S:\Capston\data3\10004_21057.pt,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10004,51033,454.75,0,S:\Capston\data3\10004_51033.pt,1,0,0,1,0,1,0,1,0,0,0,0,1,1
2,10005,18667,187.00,0,S:\Capston\data3\10005_18667.pt,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10007,47578,329.00,0,S:\Capston\data3\10007_47578.pt,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10026,29700,327.00,0,S:\Capston\data3\10026_29700.pt,1,0,1,0,1,0,0,1,0,0,1,0,0,0


In [28]:
# Specify the file path where you want to save the CSV file
csv_file_path = "S:/Capston/data2/series_meta_plus_path.csv"

# Write the DataFrame to a CSV file
series_meta_df.to_csv(csv_file_path, index=False)


In [23]:
# make a dictionary of patient_id as key and list of series as list

# Group by 'patient' column and aggregate 'series' values into lists
series_lists = series_meta_df.groupby('patient_id')['series_id'].agg(list)
paths_lists = series_meta_df.groupby('patient_id')['paths'].agg(list)

In [34]:
paths_lists[26]

[['S:/Capston/data2\\train_images/26_18881_0000.png',
  'S:/Capston/data2\\train_images/26_18881_0001.png',
  'S:/Capston/data2\\train_images/26_18881_0002.png',
  'S:/Capston/data2\\train_images/26_18881_0003.png',
  'S:/Capston/data2\\train_images/26_18881_0004.png',
  'S:/Capston/data2\\train_images/26_18881_0005.png',
  'S:/Capston/data2\\train_images/26_18881_0006.png',
  'S:/Capston/data2\\train_images/26_18881_0007.png',
  'S:/Capston/data2\\train_images/26_18881_0008.png',
  'S:/Capston/data2\\train_images/26_18881_0009.png',
  'S:/Capston/data2\\train_images/26_18881_0010.png',
  'S:/Capston/data2\\train_images/26_18881_0011.png',
  'S:/Capston/data2\\train_images/26_18881_0012.png',
  'S:/Capston/data2\\train_images/26_18881_0013.png',
  'S:/Capston/data2\\train_images/26_18881_0014.png',
  'S:/Capston/data2\\train_images/26_18881_0015.png',
  'S:/Capston/data2\\train_images/26_18881_0016.png',
  'S:/Capston/data2\\train_images/26_18881_0017.png',
  'S:/Capston/data2\\train_i

In [24]:
store_res = 128
Root = "S:/Capston/data3" 
Root2 = "S:/Capston/data4"
# Iterate over rows with tqdm progress bar
for patient, series in tqdm(series_lists.items(), total=len(series_lists)):

    # Create an empty list to store the tensors
    tensor_list = []
    for s in series:
        # series_tensor = Series_to_tensor(img_paths[index])
        series_tensor = torch.load(os.path.join(Root, str(int(patient)) + '_' + str(int(s)) + '.pt'))
        tensor_list.append(series_tensor)
    patient_tensor = np.concatenate(tensor_list, axis=2)
    patient_tensor = torch.tensor(patient_tensor, dtype=torch.float32)

    # checking tensor size and resize it if it was not in prper format
    if patient_tensor.shape != torch.Size([store_res, store_res, store_res]):
        patient_tensor = patient_tensor.unsqueeze(0).unsqueeze(0)
        patient_tensor = F.interpolate(patient_tensor, (store_res, store_res, store_res))
        patient_tensor = patient_tensor.squeeze(0).squeeze(0)

    # Check the shape of the stacked tensors
    torch.save(patient_tensor, os.path.join(Root2, str(int(patient)) + '.pt'))

100%|██████████████████████████████████████████████████████████████████████████████| 3147/3147 [48:12<00:00,  1.09it/s]


In [ ]:
store_res = 128
Root = "S:/Capston/data3" 
Root2 = "S:/Capston/data4"
c = 0
# Iterate over rows with tqdm progress bar
for patient, paths in tqdm(paths_lists.items(), total=len(paths_lists)):
    c += 1
    merged_list = [element for sublist in paths for element in sublist]
    print(len(paths[0]))
    print(len(merged_list))
    if c==5:
        break
    patient_tensor = Series_to_tensor(merged_list)
    # Check the shape of the stacked tensors
    torch.save(patient_tensor, os.path.join(Root2, str(int(patient)) + '.pt'))

In [23]:
# Making patients meta_data
Root = "S:\\Capston\\data4"
train_df["paths"] = train_df.apply(lambda row: os.path.join(Root, f"{int(row['patient_id'])}.pt"), axis=1)

In [20]:
train_df = pd.read_csv(os.path.join(data_dir,'train.csv'))

In [21]:
patients_meta_df = patients_meta_df.merge(patients_meta_df,on='patient_id')

In [24]:
train_df

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,paths
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,S:\Capston\data4\10004.pt
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\10005.pt
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\10007.pt
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\10026.pt
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1,S:\Capston\data4\10051.pt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,9951,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\9951.pt
3143,9960,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\9960.pt
3144,9961,1,0,1,0,1,0,0,1,0,0,1,0,0,0,S:\Capston\data4\9961.pt
3145,9980,1,0,1,0,1,0,0,1,0,0,0,0,1,1,S:\Capston\data4\9980.pt


In [25]:
# Specify the file path where you want to save the CSV file
csv_file_path = "S:/Capston/data2/patients_meta_plus_path.csv"

# Write the DataFrame to a CSV file
train_df.to_csv(csv_file_path, index=False)
